Objectifs du notebook :

- Nettoyer les données pour obtenir un seul fichier contenant l'échantillon qui sera utilisée.
- Vérifier la présence de valeurs manquantes ou erronées.
- S'assurer de réunir uniquement les données nécessaires pour la base de données.
- Vérifier le typage des données correspondant à la base de données prévue.

[Lien DrawDB](https://drawdb.vercel.app/editor?shareId=e6c18b8ae53063fa1dfa9cc8a849605f)

In [56]:
import matplotlib.pyplot as plt
from rich import print_json, print
from tqdm import tqdm
import pandas as pd
import glob
import os

Récupération de la liste des fichiers du dataset disponible

In [57]:
csv_files = glob.glob("./data/part_*/*.csv")
dataframes = []
print(csv_files)

[
    './data/part_9/may_july_chunk_172.csv',
    './data/part_9/may_july_chunk_171.csv',
    './data/part_9/may_july_chunk_176.csv',
    './data/part_9/may_july_chunk_168.csv',
    './data/part_9/may_july_chunk_180.csv',
    './data/part_9/may_july_chunk_179.csv',
    './data/part_9/may_july_chunk_164.csv',
    './data/part_9/may_july_chunk_162.csv',
    './data/part_9/may_july_chunk_170.csv',
    './data/part_9/may_july_chunk_167.csv',
    './data/part_9/may_july_chunk_178.csv',
    './data/part_9/may_july_chunk_177.csv',
    './data/part_9/may_july_chunk_166.csv',
    './data/part_9/may_july_chunk_175.csv',
    './data/part_9/may_july_chunk_165.csv',
    './data/part_9/may_july_chunk_169.csv',
    './data/part_9/may_july_chunk_163.csv',
    './data/part_9/may_july_chunk_174.csv',
    './data/part_9/may_july_chunk_161.csv',
    './data/part_9/may_july_chunk_173.csv',
    './data/part_1/may_july_chunk_16.csv',
    './data/part_1/may_july_chunk_13.csv',
    './data/part_1/may_july_chunk_1.csv',
    './data/part_1/may_july_chunk_17.csv',
    './data/part_1/may_july_chunk_3.csv',
    './data/part_1/may_july_chunk_20.csv',
    './data/part_1/may_july_chunk_15.csv',
    './data/part_1/may_july_chunk_7.csv',
    './data/part_1/may_july_chunk_5.csv',
    './data/part_1/may_july_chunk_6.csv',
    './data/part_1/may_july_chunk_4.csv',
    './data/part_1/may_july_chunk_14.csv',
    './data/part_1/may_july_chunk_10.csv',
    './data/part_1/may_july_chunk_8.csv',
    './data/part_1/may_july_chunk_2.csv',
    './data/part_1/may_july_chunk_9.csv',
    './data/part_1/may_july_chunk_19.csv',
    './data/part_1/may_july_chunk_18.csv',
    './data/part_1/may_july_chunk_12.csv',
    './data/part_1/may_july_chunk_11.csv'
]

Création du DataFrame final

In [58]:
x_post_df = pd.DataFrame({
    "user_id": pd.Series(dtype="int"),
    "lang": pd.Series(dtype="str"),
    "text": pd.Series(dtype="str"),
    "date": pd.Series(dtype="datetime64[ns]"),
    "like_count": pd.Series(dtype="int"),
    "reply_count": pd.Series(dtype="int"),
    "retweet_count": pd.Series(dtype="int"),
    "quote_count": pd.Series(dtype="int"),
})

print(x_post_df.dtypes)

user_id                   int64
lang                     object
text                     object
date             datetime64[ns]
like_count                int64
reply_count               int64
retweet_count             int64
quote_count               int64
dtype: object

In [ ]:
def clean_file(path, x_post_df):
    current_df = pd.read_csv(path)
    for i, row in tqdm(current_df.iterrows(), total=len(current_df), desc=f"Processing {os.path.basename(path)}"):
        try:
            # Vérification des données
            # Nous voulons :
            # - Les lignes sans valeurs nulles
            # - Les tweets en français
            # - Sans retweets ni citation de tweets.
            required_columns = ["id", "lang", "text", "date", "likeCount", "replyCount", "retweetCount", "quoteCount"]
            if any(col not in row or pd.isna(row[col]) for col in required_columns):
                continue
            if row["lang"] != "fr":
                continue
            if row["quotedTweet"] == True:
                continue
            if row["retweetedTweet"] == True:
                continue
        
            user_id = int(row["id"])
            lang = row["lang"]
            text = row["text"]
            date = row["date"]
            like_count = row["likeCount"]
            reply_count = row["replyCount"]
            retweet_count = row["retweetCount"]
            quote_count = row["quoteCount"]
            
            # Create a new row for the final dataframe
            new_row = pd.DataFrame({
                "user_id": [user_id],
                "lang": [lang],
                "text": [text],
                "date": [pd.to_datetime(date)],
                "like_count": [like_count],
                "reply_count": [reply_count],
                "retweet_count": [retweet_count],
                "quote_count": [quote_count]
            })
            
            # Append to the main dataframe
            x_post_df = pd.concat([x_post_df, new_row], ignore_index=True)
            
        except Exception as e:
            print(f"Error processing row {i} from file {os.path.basename(path)}: {str(e)}")
            continue
    
    return x_post_df
        

In [60]:
for path in csv_files[:1]:
    print(f"Processing file: {path}")
    x_post_df = clean_file(path, x_post_df)

print("[bold]x_post_df describe:[/bold]")
print(x_post_df.describe())
print("[bold]x_post_df head(5):[/bold]")
print(x_post_df.head())

Processing file: ./data/part_9/may_july_chunk_172.csv

/tmp/ipykernel_3926/3859467728.py:2: DtypeWarning: Columns (23,24,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  current_df = pd.read_csv(path)
Processing may_july_chunk_172.csv: 100%|██████████| 50000/50000 [00:02<00:00, 18921.48it/s]


x_post_df describe:

user_id                           date  like_count  reply_count  \
count  2.520000e+02                            252  252.000000   252.000000   
mean   1.812514e+18  2024-07-14 02:11:25.714285824    5.531746     0.511905   
min    1.812125e+18            2024-07-13 00:00:00    0.000000     0.000000   
25%    1.812133e+18            2024-07-13 00:00:00    0.000000     0.000000   
50%    1.812142e+18            2024-07-13 00:00:00    0.000000     0.000000   
75%    1.812151e+18            2024-07-13 00:00:00    2.000000     0.000000   
max    1.815175e+18            2024-07-21 00:00:00  376.000000    15.000000   
std    8.982858e+14                            NaN   28.314592     1.448951   

       retweet_count  quote_count  
count     252.000000   252.000000  
mean        1.107143     0.063492  
min         0.000000     0.000000  
25%         0.000000     0.000000  
50%         0.000000     0.000000  
75%         0.000000     0.000000  
max       120.000000     5.000000  
std         7.942048     0.404083

x_post_df head(5):

user_id lang  \
0  1814452713435914316   fr   
1  1814452548461572155   fr   
2  1814452503104061707   fr   
3  1814452275102036124   fr   
4  1814452130570482123   fr   

                                                text       date  like_count  \
0  @RafJacob Pensez vous que Obama pourrait se pr... 2024-07-20         0.0   
1  @DJELLE_ Ça fait juste depuis 2020 que ''Biden... 2024-07-20         1.0   
2  @Chrstianb1 @LP_LaPresse Le bordel de départ p... 2024-07-20         6.0   
3                  @TheRISEofROD Biden’s a demagogue 2024-07-20         0.0   
4  @FredericArnould Toi et les autres vous disiez... 2024-07-20         1.0   

   reply_count  retweet_count  quote_count  
0            0              0            0  
1            0              0            0  
2            1              0            0  
3            0              0            0  
4            0              0            0

In [61]:
x_post_df.to_parquet("x_post.parquet", index=False)